# 1.0 Data Exploration for Boxing Punch Predictions
This notebook explores the data collected from our boxing videos, using the pose estimation model

In [15]:
# importing necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [16]:
# set plot style
plt.style.use('seaborn-v0_8')
sns.set_context("notebook", font_scale=1.5)

In [17]:
# load the data
jab_df = pd.read_csv('../data/processed/boxing_data_cross_20240927_003752.csv')
cross_df = pd.read_csv('../data/processed/boxing_data_cross_20240927_003752.csv')
left_hook_df = pd.read_csv('../data/processed/boxing_data_left-hook_20240927_004257.csv')
right_hook_df = pd.read_csv('../data/processed/boxing_data_right-hook_20240927_004818.csv')
left_upper_df = pd.read_csv('../data/processed/boxing_data_left-uppercut_20240927_005314.csv')
right_upper_df = pd.read_csv('../data/processed/boxing_data_right-uppercut_20240927_005804.csv')
no_punches_df = pd.read_csv('../data/processed/boxing_data_no-punches_20240927_010302.csv')

In [25]:
print(f"Loaded dataset with {jab_df.shape[0]} rows and {jab_df.shape[1]} columns.")

# print jab dataset head
print(jab_df.head())

# print jab dataset info
print(jab_df.info())

# summary of jab dataset statistics
print(jab_df.describe())

Loaded dataset with 7768 rows and 135 columns.
   frame  timestamp movement_type  keypoint_0_x  keypoint_0_y  keypoint_0_z  \
0      0      0.000         cross      0.315234     -0.009300     -2.497826   
1      1      0.017         cross      0.315243      0.032026     -2.588868   
2      2      0.033         cross      0.316386      0.048633     -2.690776   
3      3      0.050         cross      0.330237      0.049839     -2.728959   
4      4      0.067         cross      0.340743      0.048670     -2.708545   

   keypoint_0_visibility  keypoint_1_x  keypoint_1_y  keypoint_1_z  \
0               0.999840      0.420325     -0.077858     -2.465585   
1               0.999834      0.418512     -0.055779     -2.568138   
2               0.999824      0.417867     -0.037747     -2.670622   
3               0.999832      0.423586     -0.033326     -2.699900   
4               0.999836      0.425525     -0.036973     -2.670472   

   keypoint_1_visibility  keypoint_2_x  keypoint_2_y  key